# 快速入门

- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/quickstart_tutorial
- 原文： https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

## 概览

- 目标：展示 PyTorch 完成典型机器学习流程的关键 API（Dataset、DataLoader、nn.Module、设备管理）。
- 本笔记结构：处理数据 → 创建模型 → 注意事项 → 练习与参考。

## 处理数据

- 核心概念：
  - `Dataset`：保存样本与标签，负责单条样本的读取与预处理（通过 `__len__` 和 `__getitem__` 实现）。
  - `DataLoader`：接收 `Dataset` 并提供可迭代的批次，支持批处理、采样、打乱、并行加载等。
- torchvision 提供常用视觉数据集（如 CIFAR、COCO、FashionMNIST 等），每个 `Dataset` 常带 `transform`（样本变换）和 `target_transform`（标签变换）。

下面示例使用 FashionMNIST 并将图片转换为 Tensor。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# 从开源数据集下载训练数据。
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从开源数据集下载测试数据。
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# 运行上述代码会下载并解压数据集。

## 创建模型

- 概念：继承 `torch.nn.Module` 来定义模型。在 `__init__` 中定义层，在 `forward` 中定义数据如何流经模型。使用 `.to(device)` 将模型移动到 `cuda`/`mps`/`cpu` 以加速训练。
- 下面给出一个简单的全连接网络示例。

In [ ]:
# 获取 cpu, gpu 或 mps 设备用于加速训练.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 定义神经网络
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

## 注意事项 / 练习 / 参考

- 注意：训练时要把输入 tensors 和模型同时移动到相同的 `device`（例如 `X = X.to(device)`）。
- 常见边界：内存不足（减小 batch_size）、num_workers 过大（降低或设为 0），MPS 可能与部分功能不兼容。